<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

:::{.callout-note}

This example code is taken from the fastai [docs](https://docs.fast.ai/tutorial.transformers.html)

:::

In [ ]:
#| include: false
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from transformers.modeling_utils import Conv1D
from fastai.text.all import *
import fastcore
from fasterai.sparse.all import *

In [ ]:
pretrained_weights = 'gpt2'
tokenizer = GPT2TokenizerFast.from_pretrained(pretrained_weights)
model = GPT2LMHeadModel.from_pretrained(pretrained_weights)

In [ ]:
path = untar_data(URLs.WIKITEXT_TINY)

In [ ]:
#| include: false
df_train = pd.read_csv(path/'train.csv', header=None)
df_valid = pd.read_csv(path/'test.csv', header=None)

In [ ]:
#| include: false
all_texts = np.concatenate([df_train[0].values, df_valid[0].values])

In [ ]:
#| include: false
class TransformersTokenizer(Transform):
    def __init__(self, tokenizer): self.tokenizer = tokenizer
    def encodes(self, x): 
        toks = self.tokenizer.tokenize(x)
        return tensor(self.tokenizer.convert_tokens_to_ids(toks))
    def decodes(self, x): return TitledStr(self.tokenizer.decode(x.cpu().numpy()))

In [ ]:
#| include: false
splits = [range_of(df_train), list(range(len(df_train), len(all_texts)))]
tls = TfmdLists(all_texts, TransformersTokenizer(tokenizer), splits=splits, dl_type=LMDataLoader)

Token indices sequence length is longer than the specified maximum sequence length for this model (4576 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
#| include: false
bs,sl = 4,256
dls = tls.dataloaders(bs=bs, seq_len=sl)

In [ ]:
#| include: false
def tokenize(text):
    toks = tokenizer.tokenize(text)
    return tensor(tokenizer.convert_tokens_to_ids(toks))

tokenized = [tokenize(t) for t in progress_bar(all_texts)]

In [ ]:
#| include: false
class TransformersTokenizer(Transform):
    def __init__(self, tokenizer): self.tokenizer = tokenizer
    def encodes(self, x): 
        return x if isinstance(x, Tensor) else tokenize(x)
        
    def decodes(self, x): return TitledStr(self.tokenizer.decode(x.cpu().numpy()))

In [ ]:
#| include: false
tls = TfmdLists(tokenized, TransformersTokenizer(tokenizer), splits=splits, dl_type=LMDataLoader)
dls = tls.dataloaders(bs=bs, seq_len=sl)

In [ ]:
#| include: false
class DropOutput(Callback):
    def after_pred(self): self.learn.pred = self.pred[0]

Let's create our fastai `Learner`.

In [ ]:
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), cbs=[DropOutput], metrics=Perplexity())

And let's try to extend a given prompt with the pretrained model.

In [ ]:
prompt = "\n = Unicorn = \n \n A unicorn is a magical creature with a rainbow tail and a horn"

In [ ]:
#| include: false
prompt_ids = tokenizer.encode(prompt)
inp = tensor(prompt_ids)[None]

In [ ]:
preds = learn.model.generate(inp, max_length=40, num_beams=5, temperature=1.5)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
tokenizer.decode(preds[0].cpu().numpy())

'\n = Unicorn = \n \n A unicorn is a magical creature with a rainbow tail and a horn on its head.\n\nA unicorn is a magical creature with a rainbow tail and a horn'

In [ ]:
learn.validate()

(#2) [3.695716619491577,40.2744255065918]

In [ ]:
learn.fit_one_cycle(1, 1e-4)

epoch,train_loss,valid_loss,perplexity,time
0,3.124115,2.844266,17.188944,07:50


In [ ]:
prompt_ids = tokenizer.encode(prompt)
inp = tensor(prompt_ids)[None]

preds = learn.model.generate(inp.cuda(), max_length=40, num_beams=5, temperature=1.5)

tokenizer.decode(preds[0].cpu().numpy())

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'\n = Unicorn = \n \n A unicorn is a magical creature with a rainbow tail and a horn on its head. It is a member of the <unk> family of <unk>,'

## Make it sparse !

Let's see now if we retrain our model, this time introducing sparsity

In [ ]:
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), cbs=[DropOutput], metrics=Perplexity())

Also, when working with text, fastai defines the number of processed batches differently, so we have to adjust our `SparsifyCallback` accordingly (luckily, fastai makes it available as the `n_batches` attribute.

@patch_to(SparsifyCallback)     
def before_fit(self):
    print(f'Pruning of {self.granularity} until a sparsity of {self.end_sparsity}%')
    self.end_epoch = self.n_epoch if self.end_epoch is None else self.end_epoch
    assert self.end_epoch <= self.n_epoch, 'Your end_epoch must be smaller than total number of epoch'

    model = self.learn.model if self.model is None else self.model # Pass a model if you don't want the whole model to be pruned
    self.sparsifier = Sparsifier(model, self.granularity, self.method, self.criteria, self.layer_type)
    self.total_iters = self.end_epoch * self.dls.n_batches
    self.start_iter = self.start_epoch * self.dls.n_batches

Let's define our `SparsifyCallback`. Let's say we want to make our model 30% sparse, by removing the highest-norm weight in each attention head.

In [ ]:
sp_cb = SparsifyCallback(sparsity=30, granularity='weight', context='local', criteria=large_final, schedule=one_cycle, layer_type=Conv1D)

We now only have to pass our callback to fastai

In [ ]:
learn.fit_one_cycle(1, 1e-4, cbs=sp_cb)

Pruning of weight until a sparsity of [30]%
Saving Weights at epoch 0


epoch,train_loss,valid_loss,perplexity,time
0,3.049835,2.859973,17.461063,09:28


Sparsity at the end of epoch 0: [30.0]%
Final Sparsity: [30.0]%
Sparsity in Conv1D 9: 30.00%
Sparsity in Conv1D 10: 30.00%
Sparsity in Conv1D 15: 30.00%
Sparsity in Conv1D 16: 30.00%
Sparsity in Conv1D 22: 30.00%
Sparsity in Conv1D 23: 30.00%
Sparsity in Conv1D 28: 30.00%
Sparsity in Conv1D 29: 30.00%
Sparsity in Conv1D 34: 30.00%
Sparsity in Conv1D 35: 30.00%
Sparsity in Conv1D 40: 30.00%
Sparsity in Conv1D 41: 30.00%
Sparsity in Conv1D 46: 30.00%
Sparsity in Conv1D 47: 30.00%
Sparsity in Conv1D 52: 30.00%
Sparsity in Conv1D 53: 30.00%
Sparsity in Conv1D 58: 30.00%
Sparsity in Conv1D 59: 30.00%
Sparsity in Conv1D 64: 30.00%
Sparsity in Conv1D 65: 30.00%
Sparsity in Conv1D 70: 30.00%
Sparsity in Conv1D 71: 30.00%
Sparsity in Conv1D 76: 30.00%
Sparsity in Conv1D 77: 30.00%
Sparsity in Conv1D 82: 30.00%
Sparsity in Conv1D 83: 30.00%
Sparsity in Conv1D 88: 30.00%
Sparsity in Conv1D 89: 30.00%
Sparsity in Conv1D 94: 30.00%
Sparsity in Conv1D 95: 30.00%
Sparsity in Conv1D 100: 30.00%
Sparsi

And we can check the predicion to the same prompt as before

In [ ]:
prompt_ids = tokenizer.encode(prompt)
inp = tensor(prompt_ids)[None]

preds = learn.model.generate(inp.cuda(), max_length=40, num_beams=5, temperature=1.5)

tokenizer.decode(preds[0].cpu().numpy())

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'\n = Unicorn = \n \n A unicorn is a magical creature with a rainbow tail and a horn @-@ like head. It is a member of the <unk> family of unicorns'

That's it ! You now have a sparse Transformer as performant as the whole model. However, this model is currently not more efficient speed and storage wise. To have such a speed-up, I suggest you to look at the [granularity](https://nathanhubens.github.io/fasterai/granularity.html) section.